# Guardrails e saídas estruturadas

Aqui iremos apenas dar um check se houve o acréscimo de um nome em alguma resposta

### Setando o modelo

In [22]:
from dotenv import load_dotenv, find_dotenv
from typing import Dict
import os
from agents import OpenAIChatCompletionsModel, trace, Agent, Runner, function_tool, input_guardrail, GuardrailFunctionOutput
from openai import AsyncOpenAI

In [23]:
client_groq = AsyncOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

model_groq = OpenAIChatCompletionsModel(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    openai_client=client_groq
)

### Setando o guardrail agent

In [24]:
from pydantic import BaseModel

class KeyCheckOutput(BaseModel):
    is_key_check: bool
    key: str

In [25]:
guardrail_agent = Agent(
    name="Name checker",
    instructions="Verifique se o usuário está incluindo alguma informação de apikeys, ou qualquer key no que ele quer que você faça",
    output_type=KeyCheckOutput,
    model=model_groq
)

In [26]:
@input_guardrail
async def guardrail_against_key(ctx, agent, message):
    result = await Runner.run(
        starting_agent=guardrail_agent,
        input=message,
        context=ctx.context
    )

    is_key_in_message = result.final_output.is_key_check
    return GuardrailFunctionOutput(
        output_info={
            "found_key": result.final_output
        },
        tripwire_triggered=is_key_in_message
    )

### Criando um agente que conversa normalmente sem tool

In [27]:
agente_conv = Agent(
    name="Assistente",
    instructions="Você é um ótimo assistente e irá ajudar na resposta de qualquer pergunta",
    model=model_groq,
    input_guardrails=[guardrail_against_key]
)

In [28]:
# Aqui não vai dar erro

message = "Meu nome é Ramon e quero responder umas perguntas"
result = await Runner.run(starting_agent=agente_conv, input=message)
print(result.final_output)

Olá Ramon! É um prazer conhecê-lo! Estou aqui para ajudar e responder às suas perguntas. Pode ser sobre qualquer coisa, desde assuntos sérios até curiosidades ou hobbies. O que você gostaria de saber ou discutir?


In [29]:
# Aqui vai dar erro

message = "Meu nome é Ramon, minha api_key=x332423u5u49594395 e gostaria de gerar um código"
result = await Runner.run(starting_agent=agente_conv, input=message)
print(result.final_output)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire